In [2]:
import os
import re
import boto3
import pandas as pd
from io import BytesIO

BUCKET = os.getenv("AWS_S3_BUCKET_NAME", "arcana-fiap")
REGION = os.getenv("AWS_REGION", "us-east-2")

def get_s3():
    return boto3.client(
        "s3",
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
        region_name=REGION,
    )

def listar_parquets(bucket=BUCKET, prefix="silver/"):
    s3 = get_s3()
    keys = []
    paginator = s3.get_paginator("list_objects_v2")
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):
            if obj["Key"].endswith(".parquet"):
                keys.append(obj["Key"])
    return keys

def nome_variavel(basename, existentes):
    """
    Converte 'meu-arquivo 1.parquet' -> 'meu_arquivo_1'
    Garante que não começa com número e não colide com nomes já usados.
    """
    name = os.path.splitext(basename)[0]
    name = re.sub(r"\W", "_", name)
    if re.match(r"^\d", name):
        name = f"df_{name}"
    original = name
    i = 1
    while name in existentes:
        name = f"{original}_{i}"
        i += 1
    return name

def carregar_parquets_em_variaveis(bucket=BUCKET, prefix="silver/", nomes=None, destino=None):
    """
    Carrega parquets do S3 e cria variáveis DataFrame com o nome do arquivo (normalizado).
    - nomes: lista de nomes (sem .parquet). Se None, carrega todos do prefixo.
    - destino: dict onde criar as variáveis (por padrão, globals()).
    Retorna um dict {nome_variavel: 's3://bucket/key'} com o mapeamento criado.
    """
    s3 = get_s3()
    destino = destino if destino is not None else globals()

    if nomes is None:
        keys = listar_parquets(bucket, prefix)
    else:
        keys = [f"{prefix}{n}.parquet" for n in nomes]

    if not keys:
        print("Nenhum parquet encontrado.")
        return {}

    criados = {}
    existentes = set(destino.keys())
    for key in keys:
        try:
            obj = s3.get_object(Bucket=bucket, Key=key)
            df = pd.read_parquet(BytesIO(obj["Body"].read()))
            var_name = nome_variavel(os.path.basename(key), existentes)
            destino[var_name] = df
            existentes.add(var_name)
            criados[var_name] = f"s3://{bucket}/{key}"
            print(f"{var_name} ← {key}  | {len(df)} linhas")
        except s3.exceptions.NoSuchKey:
            print(f"Não encontrado: s3://{bucket}/{key}")
        except Exception as e:
            print(f"Erro lendo {key}: {e}")

    return criados

In [3]:
carregar_parquets_em_variaveis(prefix="silver/")

clientes_desde ← silver/clientes_desde.parquet  | 10615 linhas
contratacoes_ultimos_12_meses ← silver/contratacoes_ultimos_12_meses.parquet  | 4314 linhas
dados_clientes ← silver/dados_clientes.parquet  | 238597 linhas
historico ← silver/historico.parquet  | 22740 linhas
mrr ← silver/mrr.parquet  | 7309 linhas
nps_relacional ← silver/nps_relacional.parquet  | 14143 linhas
nps_transacional_aquisicao ← silver/nps_transacional_aquisicao.parquet  | 178 linhas
nps_transacional_implantacao ← silver/nps_transacional_implantacao.parquet  | 662 linhas
nps_transacional_onboarding ← silver/nps_transacional_onboarding.parquet  | 208 linhas
nps_transacional_produto ← silver/nps_transacional_produto.parquet  | 113207 linhas
nps_transacional_suporte ← silver/nps_transacional_suporte.parquet  | 74794 linhas
telemetria_1 ← silver/telemetria_1.parquet  | 2535221 linhas
telemetria_10 ← silver/telemetria_10.parquet  | 1884789 linhas
telemetria_11 ← silver/telemetria_11.parquet  | 3485570 linhas
telemetria

{'clientes_desde': 's3://arcana-fiap/silver/clientes_desde.parquet',
 'contratacoes_ultimos_12_meses': 's3://arcana-fiap/silver/contratacoes_ultimos_12_meses.parquet',
 'dados_clientes': 's3://arcana-fiap/silver/dados_clientes.parquet',
 'historico': 's3://arcana-fiap/silver/historico.parquet',
 'mrr': 's3://arcana-fiap/silver/mrr.parquet',
 'nps_relacional': 's3://arcana-fiap/silver/nps_relacional.parquet',
 'nps_transacional_aquisicao': 's3://arcana-fiap/silver/nps_transacional_aquisicao.parquet',
 'nps_transacional_implantacao': 's3://arcana-fiap/silver/nps_transacional_implantacao.parquet',
 'nps_transacional_onboarding': 's3://arcana-fiap/silver/nps_transacional_onboarding.parquet',
 'nps_transacional_produto': 's3://arcana-fiap/silver/nps_transacional_produto.parquet',
 'nps_transacional_suporte': 's3://arcana-fiap/silver/nps_transacional_suporte.parquet',
 'telemetria_1': 's3://arcana-fiap/silver/telemetria_1.parquet',
 'telemetria_10': 's3://arcana-fiap/silver/telemetria_10.par

In [5]:
df_clientes_desde = clientes_desde
df_contratacoes_ultimos_12_meses = contratacoes_ultimos_12_meses
df_dados_clientes = dados_clientes
df_historico = historico
df_mrr = mrr
df_nps_relacional = nps_relacional
df_nps_transacional_aquisicao = nps_transacional_aquisicao
df_nps_transacional_implantacao = nps_transacional_implantacao
df_nps_transacional_onboarding = nps_transacional_onboarding
df_nps_transacional_produto = nps_transacional_produto
df_nps_transacional_suporte = nps_transacional_suporte
#df_telemetria_consolidado = telemetria_consolidado
df_tickets = tickets

In [20]:
df_tickets.head()

,COD_CLIENTE,NOME_GRUPO,TIPO_TICKET,STATUS_TICKET,DATA_CRIACAO,DATA_ATUALIZACAO,BK_TICKET,PRIORIDADE_TICKET
0,TFCPWG,Fábrica Consinco,task,closed,2024-11-12,2025-01-02,21787048,low
1,TB2434,PC - Financeiro,question,closed,2024-05-07,2024-05-16,20066662,normal
2,TFBYNF,TOTVS Chef Corporativo,question,closed,2024-03-19,2024-04-03,19662250,low
3,TFCNAD,PC - Financeiro,question,closed,2024-06-25,2024-07-04,20517470,normal
4,TFDJLF,AGT CLOUD RM STD,question,closed,2024-02-02,2024-02-06,19222316,low
